# Human Activity Recognition Using WiFi Signals

## Overview
Human Activity Recognition (HAR) using WiFi signals leverages the unique properties of wireless channel variations to detect different activities.

## Data Format
- **WiFi signal data** is similar to image data in structure, represented in the shape `(channels, height, width)`, but with a different interpretation:
  - `channels` → **channel**
  - `height` → **Time Steps**
  - `width` → **Antenna Pairs (transmitter-receiver combinations)**
- **Labels** represent a predefined set of classes, as is typical in classification tasks.

# Reading Data

In [1]:
!pip install -U -q gdown

import torch
import kagglehub

path = kagglehub.dataset_download("jawadkc66/q2-kaust-2025")

print("Path to dataset files:", path)

data = torch.load('/root/.cache/kagglehub/datasets/jawadkc66/q2-kaust-2025/versions/1/WiFiSensingDataset.pt')

100%|██████████| 204M/204M [00:02<00:00, 84.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/jawadkc66/q2-kaust-2025/versions/1


# Task 1: Analyze the Dataset ( Stored in `data`)

1. **Determine the number of unique labels** in the dataset.  

2. **Determine the shape of the input data** (number of samples and features).  

3. **Find the maximum value** in the dataset.  

4. **Find the minimum value** in the dataset.  

In [3]:
# 1. Determine the number of unique labels in y_train
number_of_labels = len(torch.unique(data['y_train']))
print(f"Number of unique labels: {number_of_labels}")

# 2. Determine the shape of the input data (X_train)
data_shape = data['X_train'].shape
print(f"Shape of the input data (X_train): {data_shape} (n_samples, channels, time_steps, subcarriers)")

# 3. Find the maximum value in the dataset (X_train)
maximum_value = torch.max(data['X_train'])
print(f"Maximum value in X_train: {maximum_value}")

# 4. Find the minimum value in the dataset (X_train)
minimum_value = torch.min(data['X_train'])
print(f"Minimum value in X_train: {minimum_value}")

Number of unique labels: 7
Shape of the input data (X_train): torch.Size([2500, 1, 250, 90]) (n_samples, channels, time_steps, subcarriers)
Maximum value in X_train: 1.0
Minimum value in X_train: 0.0


# Task 2: Build and Evaluate a Neural Network

1. **Design a Neural Network (Maximum 5 Layers)**  
   Build a compact neural network with no more than 5 layers. Clearly specify the type of each layer (e.g., Dense, Conv2D) and any activation functions used.

2. **Evaluate Your Model**  
   Train your network on the provided dataset and report the evaluation metrics (e.g., accuracy, loss). Discuss the performance of your model and any challenges faced during training.


In [4]:
# Task 2: Build and Evaluate a Neural Network

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Create a simple neural network (maximum 5 layers)
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleNN, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Prepare the dataset
X_train = data['X_train'].reshape(data['X_train'].shape[0], -1)  # Flatten for fully connected layers
y_train = data['y_train'].long()
X_test = data['X_test'].reshape(data['X_test'].shape[0], -1)
y_test = data['y_test'].long()

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize the model
input_size = X_train.shape[1]
num_classes = number_of_labels
model = SimpleNN(input_size, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 10
for epoch in range(epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Evaluate the model
with torch.no_grad():
    test_outputs = model(X_test)
    predictions = torch.argmax(test_outputs, axis=1)
    accuracy = (predictions == y_test).sum().item() / y_test.size(0)

print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/10, Loss: 1.6662
Epoch 2/10, Loss: 1.4864
Epoch 3/10, Loss: 1.9849
Epoch 4/10, Loss: 1.2083
Epoch 5/10, Loss: 0.4782
Epoch 6/10, Loss: 1.7715
Epoch 7/10, Loss: 1.2632
Epoch 8/10, Loss: 0.7357
Epoch 9/10, Loss: 1.3324
Epoch 10/10, Loss: 0.7918
Test Accuracy: 52.60%
